In [104]:
import json 
import os 
from collections import defaultdict
import inflection as inf
import random

annotation_dir = "annotations/smallv1"
annotation_filename = "iccv-real-small-objdet.json"
annotation_path = os.path.join(annotation_dir, annotation_filename)

val_filepath = "annotations/annotations_val.json"


In [105]:
determiners = ["a", "an", "all", "any", "every", "my", "your", "this", "that", "these", "those", "some", "many", "few", "both", "neither", "little", "much", "either", "our", "no", "several", "half", "each", "the"]

In [127]:
data = json.load(open(annotation_path))
val_data = json.load(open(val_filepath))

val_categories = val_data["categories"]

ann_map = defaultdict(lambda: defaultdict(list))
cat_map = defaultdict(int)
cat_id_map = defaultdict(int)
img_map = defaultdict(int)
categories = data["categories"]

for cat in categories:
    name = cat["name"]
    if name.split()[0] in determiners:
        name = " ".join(name.split()[1:])
    else: 
        name = cat["name"]
    for c in val_categories:
        if name == c["name"]:
            cat_id_map[cat["id"]] = c["id"]

print(cat_id_map)

for i, img in enumerate(data["images"]):
    img_map[img["id"]] = i
    data["images"][i]["file_name"] = "images/" + img["file_name"]

for i, cat in enumerate(categories):
    new_id = cat_id_map[cat["id"]]
    categories[i]["id"] = new_id
    for c in val_categories: 
        if new_id == c["id"]:
            cat_map[new_id] = c["name"]

for ann in data["annotations"]:
    ann["category_id"] = cat_id_map[ann["category_id"]]
    ann_map[ann["image_id"]][ann["category_id"]].append(ann)

defaultdict(<class 'int'>, {1: 11, 2: 0, 3: 1, 4: 13, 6: 6, 7: 6, 8: 1, 10: 2, 11: 7, 14: 13, 18: 3, 22: 6, 25: 3, 27: 3, 28: 3, 35: 1, 36: 3, 41: 0, 45: 6, 46: 8, 48: 7, 52: 0, 53: 15, 55: 0, 57: 14, 58: 4, 61: 8, 65: 4, 66: 9, 71: 7, 76: 5, 78: 13, 79: 3, 81: 0, 83: 2, 85: 9, 86: 0, 87: 2, 88: 3, 89: 4, 90: 5, 91: 6, 92: 7, 93: 8, 94: 9, 95: 10, 96: 11, 97: 12, 98: 13, 99: 14, 100: 15, 101: 1, 102: 1, 103: 11, 104: 2, 105: 3, 106: 0, 107: 0, 110: 3, 111: 3, 112: 0, 115: 0, 117: 3, 118: 3, 119: 0, 120: 0, 121: 0, 122: 0})


In [128]:
cat_map

defaultdict(int,
            {11: 'cranberry juice',
             0: 'apple',
             1: 'onion',
             13: 'papaya juice',
             6: 'cucumber',
             2: 'avocado',
             7: 'lemon',
             3: 'orange',
             8: 'garlic',
             15: 'tray',
             14: 'vegetable juice',
             4: 'egg',
             9: 'banana',
             5: 'carrot',
             10: 'grape juice',
             12: 'blueberry juice'})

In [129]:
data["images"]

[{'id': 1,
  'datatorch_id': '445efbb4-e341-4c7d-9d23-72581c1936e8',
  'storage_id': '1c8751ef-dfc8-4b41-9f8b-c5dd5ff755e2',
  'path': '000029.jpg',
  'width': 256,
  'height': 256,
  'file_name': 'images/000029.jpg',
  'metadata': {},
  'date_captured': '2023-02-27T09:31:11.322Z'},
 {'id': 2,
  'datatorch_id': '151483c2-6fea-427a-9dad-fcc22205bdfd',
  'storage_id': '1c8751ef-dfc8-4b41-9f8b-c5dd5ff755e2',
  'path': '000000.jpg',
  'width': 256,
  'height': 256,
  'file_name': 'images/000000.jpg',
  'metadata': {},
  'date_captured': '2023-02-27T09:31:11.260Z'},
 {'id': 3,
  'datatorch_id': '6d40c27d-c79c-4e90-84cf-9abce1bf3175',
  'storage_id': '1c8751ef-dfc8-4b41-9f8b-c5dd5ff755e2',
  'path': '000030.jpg',
  'width': 256,
  'height': 256,
  'file_name': 'images/000030.jpg',
  'metadata': {},
  'date_captured': '2023-02-27T09:31:11.244Z'},
 {'id': 4,
  'datatorch_id': 'f9a286aa-b0aa-4f29-8991-a44b28dfa6d2',
  'storage_id': '1c8751ef-dfc8-4b41-9f8b-c5dd5ff755e2',
  'path': '000015.jpg',

In [130]:
images = []
annotations = []
input_oracle_annotations = data["annotations"]

det_counts = defaultdict(int)
for key in ann_map.keys(): 
    anns = ann_map[key]
    img = data["images"][img_map[key]]
    for cat_id in anns.keys():
        cat_name = cat_map[cat_id]
        n_below = 0 
        n_above = 0
        for ann in anns[cat_id]:
            x, y, w, h = ann["bbox"]
            mid_x = x + w/2
            mid_y = y + h/2

            if mid_y > 128:
                n_above += 1
            else:
                n_below += 1

        if cat_name == "tray": 
            continue
        if not cat_name.split()[-1] == "juice":
            if cat_name[0] in ["a", "e", "i", "o", "u"]:
                img_copy = img.copy()
                img_copy["id"] = len(images)
                img_copy["caption"] = f"an {cat_name}"
                images.append(img_copy)
            
                ann_copy = anns[cat_id][0].copy()
                ann_copy["id"] = len(annotations)
                ann_copy["image_id"] = img_copy["id"]
                annotations.append(ann_copy)

                det_counts["an"] += 1
            else: 
                img_copy = img.copy()
                img_copy["id"] = len(images)
                img_copy["caption"] = f"a {cat_name}"
                images.append(img_copy)
            
                ann_copy = anns[cat_id][0].copy()
                ann_copy["id"] = len(annotations)
                ann_copy["image_id"] = img_copy["id"]
                annotations.append(ann_copy)

                det_counts["a"] += 1

            img_copy = img.copy()
            img_copy["id"] = len(images)
            img_copy["caption"] = f"any {inf.singularize(cat_name)}"
            images.append(img_copy)
        
            ann_copy = anns[cat_id][0].copy()
            ann_copy["id"] = len(annotations)
            ann_copy["image_id"] = img_copy["id"]
            annotations.append(ann_copy)

            det_counts["any"] += 1
                
            if len(anns[cat_id]) == 1:
                img_copy = img.copy()
                img_copy["id"] = len(images)
                img_copy["caption"] = f"the {cat_name}"
                images.append(img_copy)
            
                ann_copy = anns[cat_id][0].copy()
                ann_copy["id"] = len(annotations)
                ann_copy["image_id"] = img_copy["id"]
                annotations.append(ann_copy)

                det_counts["the"] += 1

            if len(anns[cat_id]) >= 8:
                img_copy = img.copy()
                img_copy["id"] = len(images)
                img_copy["caption"] = f"many {inf.pluralize(cat_name)}"
                images.append(img_copy)
            
                count = min(len(anns[cat_id]), random.randint(8,9))
                for ann in anns[cat_id][:count]:
                    ann_copy = ann.copy()
                    ann_copy["id"] = len(annotations)
                    ann_copy["image_id"] = img_copy["id"]
                    annotations.append(ann_copy)

                det_counts["many"] += 1

            if len(anns[cat_id]) >= 6:
                img_copy = img.copy()
                img_copy["id"] = len(images)
                img_copy["caption"] = f"several {inf.pluralize(cat_name)}"
                images.append(img_copy)
            
                count = min(len(anns[cat_id]), random.randint(6,7))
                for ann in anns[cat_id][:count]:
                    ann_copy = ann.copy()
                    ann_copy["id"] = len(annotations)
                    ann_copy["image_id"] = img_copy["id"]
                    annotations.append(ann_copy)
                
                det_counts["several"] += 1

            if len(anns[cat_id]) >= 4:
                img_copy = img.copy()
                img_copy["id"] = len(images)
                img_copy["caption"] = f"some {inf.pluralize(cat_name)}"
                images.append(img_copy)
            
                count = min(len(anns[cat_id]), random.randint(4,5))
                for ann in anns[cat_id][:count]:
                    ann_copy = ann.copy()
                    ann_copy["id"] = len(annotations)
                    ann_copy["image_id"] = img_copy["id"]
                    annotations.append(ann_copy)
                
                det_counts["some"] += 1
            
            if len(anns[cat_id]) >= 3:
                img_copy = img.copy()
                img_copy["id"] = len(images)
                img_copy["caption"] = f"few {inf.pluralize(cat_name)}"
                images.append(img_copy)
            
                count = min(len(anns[cat_id]), random.randint(2,3))
                for ann in anns[cat_id][:count]:
                    ann_copy = ann.copy()
                    ann_copy["id"] = len(annotations)
                    ann_copy["image_id"] = img_copy["id"]
                    annotations.append(ann_copy)
                det_counts["few"] += 1

                img_copy = img.copy()
                img_copy["id"] = len(images)
                img_copy["caption"] = f"every {inf.singularize(cat_name)}"
                images.append(img_copy)
            
                for ann in anns[cat_id]:
                    ann_copy = ann.copy()
                    ann_copy["id"] = len(annotations)
                    ann_copy["image_id"] = img_copy["id"]
                    annotations.append(ann_copy)
                det_counts["every"] += 1
            
            if len(anns[cat_id]) == 2:
                img_copy = img.copy()
                img_copy["id"] = len(images)
                img_copy["caption"] = f"both {inf.pluralize(cat_name)}"
                images.append(img_copy)
            
                for ann in anns[cat_id]:
                    ann_copy = ann.copy()
                    ann_copy["id"] = len(annotations)
                    ann_copy["image_id"] = img_copy["id"]
                    annotations.append(ann_copy)

                det_counts["both"] += 1

                img_copy = img.copy()
                img_copy["id"] = len(images)
                img_copy["caption"] = f"neither {inf.pluralize(cat_name)}"
                images.append(img_copy)
            
                for ann in anns[cat_id]:
                    ann_copy = ann.copy()
                    ann_copy["id"] = len(annotations)
                    ann_copy["image_id"] = img_copy["id"]
                    annotations.append(ann_copy)

                det_counts["neither"] += 1

                img_copy = img.copy()
                img_copy["id"] = len(images)
                img_copy["caption"] = f"either {inf.singularize(cat_name)}"
                images.append(img_copy)
                
                for ann in anns[cat_id][:1]:
                    ann_copy = ann.copy()
                    ann_copy["id"] = len(annotations)
                    ann_copy["image_id"] = img_copy["id"]
                    annotations.append(ann_copy)

                det_counts["either"] += 1

            if len(anns[cat_id]) >=2:
                img_copy = img.copy()
                img_copy["id"] = len(images)
                img_copy["caption"] = f"each {inf.singularize(cat_name)}"
                images.append(img_copy)
            
                for ann in anns[cat_id]:
                    ann_copy = ann.copy()
                    ann_copy["id"] = len(annotations)
                    ann_copy["image_id"] = img_copy["id"]
                    annotations.append(ann_copy)
                det_counts["each"] += 1

                img_copy = img.copy()
                img_copy["id"] = len(images)
                img_copy["caption"] = f"all {inf.pluralize(cat_name)}"
                images.append(img_copy)
            
                for ann in anns[cat_id]:
                    ann_copy = ann.copy()
                    ann_copy["id"] = len(annotations)
                    ann_copy["image_id"] = img_copy["id"]
                    annotations.append(ann_copy)
                det_counts["all"] += 1

                img_copy = img.copy()
                img_copy["id"] = len(images)
                img_copy["caption"] = f"no {inf.pluralize(cat_name)}"
                images.append(img_copy)
            
                for ann in anns[cat_id]:
                    ann_copy = ann.copy()
                    ann_copy["id"] = len(annotations)
                    ann_copy["image_id"] = img_copy["id"]
                    annotations.append(ann_copy)
                det_counts["no"] += 1

            if len(anns[cat_id]) % 2 == 0: 
                img_copy = img.copy()
                img_copy["id"] = len(images)
                img_copy["caption"] = f"half {inf.pluralize(cat_name)}"
                images.append(img_copy)
            
                for ann in anns[cat_id][:len(anns[cat_id])//2]:
                    ann_copy = ann.copy()
                    ann_copy["id"] = len(annotations)
                    ann_copy["image_id"] = img_copy["id"]
                    annotations.append(ann_copy)

                det_counts["half"] += 1

            if n_above >= 3: 
                img_copy = img.copy()
                img_copy["id"] = len(images)
                img_copy["caption"] = f"those {inf.pluralize(cat_name)}"
                images.append(img_copy)
            
                count = min(len(anns[cat_id]), random.randint(2,4))
                for ann in anns[cat_id][:count]:
                    ann_copy = ann.copy()
                    ann_copy["id"] = len(annotations)
                    ann_copy["image_id"] = img_copy["id"]
                    annotations.append(ann_copy)
                det_counts["those"] += 1
            
            if n_below >= 3:
                img_copy = img.copy()
                img_copy["id"] = len(images)
                img_copy["caption"] = f"these {inf.pluralize(cat_name)}"
                images.append(img_copy)
            
                count = min(len(anns[cat_id]), random.randint(2,4))
                for ann in anns[cat_id][:count]:
                    ann_copy = ann.copy()
                    ann_copy["id"] = len(annotations)
                    ann_copy["image_id"] = img_copy["id"]
                    annotations.append(ann_copy)
                det_counts["these"] += 1  

            if n_above == 1:
                img_copy = img.copy()
                img_copy["id"] = len(images)
                img_copy["caption"] = f"that {cat_name}"
                images.append(img_copy)
            
                ann_copy = anns[cat_id][0].copy()
                ann_copy["id"] = len(annotations)
                ann_copy["image_id"] = img_copy["id"]
                annotations.append(ann_copy)
                det_counts["that"] += 1

            if n_below == 1:
                img_copy = img.copy()
                img_copy["id"] = len(images)
                img_copy["caption"] = f"this {cat_name}"
                images.append(img_copy)
            
                ann_copy = anns[cat_id][0].copy()
                ann_copy["id"] = len(annotations)
                ann_copy["image_id"] = img_copy["id"]
                annotations.append(ann_copy)
                det_counts["this"] += 1   



In [131]:
print("number of images: ", len(images))
print("number of annotations: ", len(annotations), "\n")
print("num of determiners in dataset: \n", det_counts, "\n")
det_set = set(det_counts.keys())
print("determiners not included: \n", set(determiners).difference(det_set))


number of images:  475
number of annotations:  1619 

num of determiners in dataset: 
 defaultdict(<class 'int'>, {'an': 40, 'any': 60, 'several': 12, 'some': 22, 'few': 30, 'every': 30, 'each': 41, 'all': 41, 'no': 41, 'half': 23, 'those': 15, 'many': 8, 'these': 15, 'both': 11, 'neither': 11, 'either': 11, 'a': 20, 'the': 19, 'that': 14, 'this': 11}) 

determiners not included: 
 {'little', 'our', 'my', 'your', 'much'}


In [126]:
new_annot = {
    "categories": val_categories,
    "images": images,
    "annotations": annotations,
    "input_oracle_annotations": data["annotations"]
}

json.dump(new_annot, open("annotations/smallv1/generated.json", "w"))